In [1]:
%load_ext autoreload
%autoreload 2

This notebook is meant to demo a few dev techniques when tackling a new problem. 

It involves: TDD, typing, scaffolding, protocols, facades, repository pattern,... 

## The problem

The original problem: **As a user, I can tag (recording) sessions, view my tags, and retrieve the sessions for a given tag.**

Questions:

- Where do I store the tag/sessions info?
- How (what data structure, what am I actually storing)?

Answers:

Shouldn't start with those questions. Those questions have to do with specifics. Of course, we'll need to make choices about those questions eventually, but we'll produce a brittle code-base if we don't first think of abstractions.

Let's first think of a more general expression of what we're actually trying to do, and find a less specific vocabulary to describe it. We can (maybe even should) reintroduce the domain-specific vocabulary, as a layer on top of the more general mechanism, but we want to give ourselves a chance of knowing what the more abstract problem pattern is.

The transformed, more general, so reusable, problem: **Same as above but replace "tag" with "group" and "sessions" with "items".**

Then:

* First look at what makes sense in the domain/interface -- Express this with types and tests

* Then implement the interface, with a backend that corresponds to the current constraints

## A little digression on scaffolding

In [4]:
from typing import MutableMapping, Iterable, Any, NewType, Callable

Group = NewType('Group', str)
Item = NewType('Item', Any)


What functionality do we want around groups and their items?

Let's express this through type annotations of some functions (which we'll encapsulate in a `Groups` class.

In [5]:
class GroupsDacc:
    add_items_to_group: Callable[[Iterable[Item], Group], Any]
    list_groups: Callable[[], Iterable[Group]]
    items_for_group: Callable[[Group],Iterable[Item]]
    

Note that this should now be sufficient to generate a scaffold of what we need. 

We'll do it in two ways: By (dynamically) creating a `typing.protocol` describing a concrete `Groups` object we could implement in the future, and by generating (the code string for) a concrete (but empty) such `Groups` class.

In [6]:
import i2

from meshed.scrap.annotations_to_meshes import (
    func_types_to_scaffold, 
    func_types_to_protocol
)

# Groups.__annotations__ is a {name: func_annotation, ...} dict
# We can make a protocol from that
GroupsProtocol = func_types_to_protocol(GroupsDacc.__annotations__)

# See that GroupsProtocol has methods for each item of the 
# Groups.__annotations__ dict. Each method bares a signature compatible with 
# the annotations.
i2.Sig(GroupsProtocol.add_items_to_group)
# <Sig (self, iterable: Iterable[__main__.Item], group: __main__.Group) -> Any>

<Sig (self, iterable: Iterable[__main__.Item], group: __main__.Group) -> Any>

In [7]:
# We can also 
print(func_types_to_scaffold(GroupsDacc.__annotations__))


class GeneratedClass:
    def add_items_to_group(self, iterable: Iterable, group: Group) -> Any:
    	pass

    def list_groups(self) -> Iterable:
    	pass

    def items_for_group(self, group: Group) -> Iterable:
    	pass



## TDD: Tests that describe the behavior we want

In [8]:
from typing import MutableMapping, Iterable, Any, NewType, Callable, Protocol

Group = NewType('Group', str)
Item = NewType('Item', Any)


class GroupsProtocol(Protocol):
    def add_items_to_group(self, iterable: Iterable, group: Group) -> Any:
        """Add one or several items to a group"""

    def list_groups(self) -> Iterable:
        """List group names"""

    def items_for_group(self, group: Group) -> Iterable:
        """List the items in a group"""
    
    
def test_groups(groups: GroupsProtocol):
    # the following assertion isn't part of the behavior we want -- just a condition we'll 
    # need to be able to conduct our test: Namely, that our collection of groups/items is empty.
    assert list(groups.list_groups()) == []  # make sure test is well setup
    
    groups.add_items_to_group('group_a', 'item_1', 'item_2')
    assert sorted(groups.list_groups()) == ['group_a']
    
    groups.add_items_to_group('group_b', 'item_3')
    assert sorted(groups.list_groups()) == ['group_a', 'group_b']
    assert sorted(groups.items_for_group('group_a')) == ['item_1', 'item_2']
    

Now we'll implement two concrete `GroupsDacc`, using a store, a `MutableMapping`, as a back-end so as to keep the persistance concern still separate. 

The idea is: As long as we provide our concrete persister with the right `MutableMapping` facade (with a minimum of specifics/semantics such as what the keys and values are meant to be), we should have a working object.

The two `GroupsDacc` options will differ on the particulars of the store. 
- In the first, we'll assume the store has items as keys and groups as values. 
- In the second we'll assume the groups are the keys, and values are sets of items of that group.

## Concrete GroupsDacc (option 1): ItemGroupDacc

In [15]:
from typing import MutableMapping, Iterable, Any, NewType
from dataclasses import dataclass

Group = NewType('Group', str)
Item = NewType('Item', Any)
ItemGroupPairs = NewType('ItemGroupPairs', MutableMapping[Item, Group])

@dataclass
class ItemGroupDacc:
    store: ItemGroupPairs
        
    def add_items_to_group(self, group: Group, *items: Iterable[Item]) -> Any:
        for item in items:
            self.store[item] = group

    def list_groups(self) -> Iterable[Group]:
        return set(self.store.values())

    def items_for_group(self, group: Group) -> Iterable[Item]:
        # TODO: Exxpress this filtering in such a way that will allow us to take advantage of DB specifics
        #  (e.g., passing on the filtering to the DB instead of filtering in python itself)
        return (item for item, group_ in self.store.items() if group_ == group)
    

In [16]:
store = dict()
test_groups(ItemGroupDacc(store))  # a dict works!

## Concrete GroupsDacc (option 2): GroupSetsDacc

In [17]:
from typing import MutableMapping, Iterable, Any, NewType, Set
from dataclasses import dataclass

Group = NewType('Group', str)
Item = NewType('Item', Any)
GroupSets = NewType('GroupSets', MutableMapping[Group, Set[Item]])

@dataclass
class GroupSetsDacc:
    store: GroupSets
        
    def add_items_to_group(self, group: Group, *items: Iterable[Item]) -> Any:
        self.store[group] |= set(items)

    def list_groups(self) -> Iterable[Group]:
        return set(self.store)
            
    def items_for_group(self, group: Group) -> Iterable[Item]:
        return self.store[group]
    

In [18]:
from collections import defaultdict

store = defaultdict(set)
test_groups(GroupSetsDacc(store))  # a defaultdict(set) works as a store!

## Actual persisting stores using mongo

In [19]:
# from mongodol.base import MongoClient

### ItemGroupPairs (for ItemGroupDacc)

Options for implementing:

```
s[item] = group
```

Option 1: But here we'd need to produce the ID on write

```
--> {'_id': ID, 'group': group', 'item': item}
```

Option 2: But we need to allow re-writes on `_id`

```
--> {'_id': item, 'group': group}
```


In [23]:
from operator import itemgetter

from dol import wrap_kvs, Pipe
from mongodol.stores import MongoStore


# To be able to overwrite an existing (item, group) pair (by default MongoStore doesn't allow it)
def delete_if_exists(self, k, v):
    if k in self:
        del self[k]
    return v


trans = Pipe(
    wrap_kvs(
        key_of_id=itemgetter('_id'), 
        id_of_key=lambda x: {'_id': x}, 
        obj_of_data=itemgetter('group'),
        data_of_obj=lambda x: {'group': x}, 
        preset=delete_if_exists,
    )
)

@trans
class GroupStore(MongoStore):
    """To group items"""
    def __init__(self,
        db_name='scrap',
        collection_name='group_items',
        mongo_client_kwargs=None,
    ):
        super().__init__(
            db_name=db_name,
            collection_name=collection_name,
            key_fields=['_id'],
            data_fields=['group'],
            mongo_client_kwargs=mongo_client_kwargs
        )
    
m = GroupStore()
list(m)

[]

In [24]:
store = GroupStore()

# empty the store
for k in store: 
    del store[k]

test_groups(ItemGroupDacc(store))

### GroupSets (for GroupSetsDacc)

Options for implementing:

```
s[group] |= items
```

Option 1: But here we'd need to produce the ID on write

```
--> {'_id': group, 'items': items}
```

Option 2: But we need to allow re-writes on `_id`

```
--> {'_id': ID, 'group': group, 'item': item}  # (group, items) -> (group, item_1), (group, item_2), ...
```


In [63]:
from operator import itemgetter

from dol import wrap_kvs, Pipe
from mongodol.stores import MongoStore


# To be able to overwrite an existing (item, group) pair (by default MongoStore doesn't allow it)
def delete_if_exists(self, k, v):
    if k in self:
        del self[k]
    return v


trans = Pipe(
    wrap_kvs(
        key_of_id=itemgetter('_id'), 
        id_of_key=lambda x: {'_id': x}, 
        obj_of_data=Pipe(itemgetter('items'), set),
        data_of_obj=lambda x: {'items': list(x) if not isinstance(x, str) else [x]}, 
#         preset=delete_if_exists,
    )
)

@trans
class ItemsStore(MongoStore):
    """To group items"""
    def __init__(self,
        db_name='scrap',
        collection_name='items_group',
        mongo_client_kwargs=None,
    ):
        super().__init__(
            db_name=db_name,
            collection_name=collection_name,
            key_fields=['_id'],
            data_fields=['items'],
            mongo_client_kwargs=mongo_client_kwargs
        )
        
    def __missing__(self, k):
        return {'items': []}
    

In [66]:
store = ItemsStore()

# empty the store
for k in store: 
    del store[k]

test_groups(GroupSetsDacc(store))